In [8]:
import json
import requests


video = {
    # "path": "/workspaces/aana_sdk/aana/tests/files/videos/physicsworks.webm"
    "url": "https://mobius-public.s3.eu-west-1.amazonaws.com/squirrel.mp4",
    "media_id": "squirrel.mp4",
    # "url": "https://www.youtube.com/watch?v=yModCU1OVHY",
    # "media_id": "yModCU1OVHY",
    # "url": "https://www.youtube.com/watch?v=77ZF50ve6rs",
    # "media_id": "77ZF50ve6rs",
    # "url": "https://www.youtube.com/watch?v=wxN1T1uxQ2g",
    # "media_id": "wxN1T1uxQ2g",
}

In [9]:
data = {
    "video_params": {
        "fast_mode_enabled": True,
    },
    "video": video,
}
url = "http://127.0.0.1:8001/video/index_stream"
response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
# visuals = response.json()
for chunk in response.iter_content(chunk_size=None):
    print(chunk)
    json_data = json.loads(chunk)

    if "error" in json_data:
        print(json_data["error"])
        print(json_data["message"])
        print(json_data["stacktrace"])
        break
    else:
        segments = json_data.get("captions")

    if segments:
        print(segments)

b'{"transcription":{"text":""},"segments":[],"info":{"language":"silence","language_confidence":1.0}}'
b'{"timestamps":[0.0,3.0399999618530273,6.079999923706055,9.119999885559082],"captions":["a squirrel is sitting on top of a table","a squirrel sitting on top of a table","a squirrel is sitting on a table with its mouth open","a squirrel is sitting on top of a table"]}'
['a squirrel is sitting on top of a table', 'a squirrel sitting on top of a table', 'a squirrel is sitting on a table with its mouth open', 'a squirrel is sitting on top of a table']
b'{"caption_ids":[33,34,35,36],"transcription_id":8}'


In [10]:
media_id = video["media_id"]

question = (
    "What is happening in this video? "
    "Focus on highlighting the story, the narrative, emotionally and thematically significant aspects. "
    "Do not use bulleted list in your answer."
)

In [11]:
data = {
    "question": question,
    "media_id": media_id,
    "sampling_params": {
        "temperature": 0.9,
        "max_tokens": 1024,
    },
}

url = "http://127.0.0.1:8001/video/chat_stream"

response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
text = ""
for chunk in response.iter_content(chunk_size=None):
    chunk_dict = json.loads(chunk)
    if "error" in chunk_dict:
        print(chunk_dict["error"])
        print(chunk_dict["message"])
        print(chunk_dict["stacktrace"])
        break
    token = chunk_dict["completion"]
    text += token
    print(token, end="")

  Based on the provided video timeline and title, the story that emerges is one of a curious and playful squirrel exploring its surroundings. The video opens with the squirrel sitting on top of a table, and the visual caption and audio transcript both indicate that the animal is sitting with its mouth open, suggesting that it is feeling carefree and relaxed.
Throughout the video, the squirrel goes on to explore various objects and surfaces in its environment, using its agility and curiosity to manipulate its surroundings. The visual caption and audio transcript both suggest that the squirrel is engaged in playful and curious behavior, such as playing with a ball or leaping from one surface to another.
Overall, the video tells a story of a playful and curious squirrel exploring its environment, and the visual caption and audio transcript both work together to convey the sense of fun and adventure that the animal is experiencing.